In [ ]:
!pip install pandas recordlinkage tabulate sklearn py_stringmatching matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import time
from pathlib import Path
import json
import recordlinkage
from tabulate import tabulate
from recordlinkage.base import BaseCompareFeature
from recordlinkage.index import BaseIndexAlgorithm
import os
import warnings
import pprint
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from py_stringmatching import QgramTokenizer
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from google.colab import drive

drive.mount("/content/gdrive/")
warnings.filterwarnings("ignore")

path = "/content/gdrive/MyDrive/Colab Notebooks"
path_music = path+"/Data Inputs/music_brainz_20k/"

Mounted at /content/gdrive/


In [ ]:
music_columns = ["album","length","artist","year","title","language"]
music_dataset = pd.read_csv(path_music + "music_2mil.csv",encoding = 'UTF-8',usecols=music_columns)

In [ ]:
music_dataset.head()

,title,length,artist,album,year,language
0,L'enfant aux yeux d'Italie,219800,Daniel Balavoine,De vous à elle en passant par moi,1975,French
1,Mustard Gas,129000,Action Painting!,There and Back Again Lane,1995,English
2,Angels From Heaven (radio edit) (The Next Chap...,03:09,MaIrio Lopez,The Next Chapter,2002,NaN
3,Kathy Troccoli - Your Grace,166,NaN,Comfort,05,English
4,"Brandenburg Concerto No. 3 in G major, BWV 1048",816866,Johann Sebastian Bach,"Brandenburg Concerti (Mainz Chamber Orchestra,...",1988,English


In [ ]:
no_lg_detected = music_dataset[music_dataset["language"].isna()]

In [ ]:
music_columns_no_lg = no_lg_detected[["album","artist","title","language"]]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
music_columns_no_lg[music_columns_no_lg.isna().any(axis=1)]

,album,artist,title,language
2,The Next Chapter,MaIrio Lopez,Angels From Heaven (radio edit) (The Next Chap...,NaN
6,First Down,Ernie Stadler Jazz Quintet,Well You Needn't (First Down),NaN
14,Flow Goes The Universe,Laraaji,Being Here (Flow Goes The Universe),NaN
27,10cm,Anemone,10c,NaN
30,Tabu Recz,Tabu Recz,"Du Ikk' Sej, Din Pik Er Ikk' Lang, Vi Skyder D...",NaN
...,...,...,...,...
1937488,Aaspelunker,Wytchyker,Raised By Wolves (Aaspelunker),NaN
1937493,Bridget Jones's Diary 2: More Music From the M...,Mary Chapin Carpenter,Passionate Kisses (Bridget Jones's Diary 2: Mo...,NaN
1937494,NaN,Gorp,Leaders of Faith - Deficite & Defiance,NaN
1937496,Christmas Stars,NaN,Luciano Pavarotti - Gesu Redeemer,NaN


In [ ]:
print(tabulate(music_columns_no_lg.head(20), headers=music_columns_no_lg.columns.tolist(), tablefmt="pretty"))

+----+------------------------------------------------------------------------------------+----------------------------+--------------------------------------------------------------------------------------------------------------------------------+----------+
|    |                                       album                                        |           artist           |                                                             title                                                              | language |
+----+------------------------------------------------------------------------------------+----------------------------+--------------------------------------------------------------------------------------------------------------------------------+----------+
| 2  |                                  The Next Chapter                                  |        MaIrio Lopez        |                                       Angels From Heaven (radio edit) (The Next Chapter)        

In [ ]:
music_columns_no_lg = music_columns_no_lg.apply(lambda x: x.astype(str).str.lower())
import string
import re

In [ ]:
chars = re.escape(string.punctuation)
music_columns_no_lg["album"] = music_columns_no_lg["album"].apply(lambda x: re.sub(' +', ' ', re.sub(r'['+chars+']', ' ',x)))
music_columns_no_lg["artist"] = music_columns_no_lg["artist"].apply(lambda x: re.sub(' +', ' ', re.sub(r'['+chars+']', ' ',x)))
music_columns_no_lg["title"] = music_columns_no_lg["title"].apply(lambda x: re.sub(' +', ' ', re.sub(r'['+chars+']', ' ',x)))

In [ ]:
print(tabulate(music_columns_no_lg.head(20), headers=music_columns_no_lg.columns.tolist(), tablefmt="pretty"))

+----+-------------------------------------------------------------------------------+----------------------------+------------------------------------------------------------------------------------------------------------------------+----------+
|    |                                     album                                     |           artist           |                                                         title                                                          | language |
+----+-------------------------------------------------------------------------------+----------------------------+------------------------------------------------------------------------------------------------------------------------+----------+
| 2  |                               the next chapter                                |        mairio lopez        |                                     angels from heaven radio edit the next chapter                                     |   nan    |
| 6  |  

In [ ]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from langdetect import detect
from langdetect import DetectorFactory
DetectorFactory.seed = 0

def detect_language(row):
    try:
      return detect(row)
    except Exception as ex:
      return str(ex)
music_columns_no_lg = music_columns_no_lg.head(2000)
music_columns_no_lg["combined_col"]=music_columns_no_lg[["album", "title"]].apply(" ".join, axis=1)
music_columns_no_lg["language"]=music_columns_no_lg["combined_col"].apply(lambda x: detect_language(x))
music_columns_no_lg.drop(inplace=True,columns=["combined_col"])

In [ ]:
print(tabulate(music_columns_no_lg.head(20), headers=music_columns_no_lg.columns.tolist(), tablefmt="pretty"))

+----+-------------------------------------------------------------------------------+----------------------------+------------------------------------------------------------------------------------------------------------------------+----------+
|    |                                     album                                     |           artist           |                                                         title                                                          | language |
+----+-------------------------------------------------------------------------------+----------------------------+------------------------------------------------------------------------------------------------------------------------+----------+
| 2  |                               the next chapter                                |        mairio lopez        |                                     angels from heaven radio edit the next chapter                                     |    en    |
| 6  |  

In [ ]:
music_columns_no_lg["language"].unique()

array(['en', 'vi', 'no', 'de', 'it', 'cs', 'fr', 'es', 'ru', 'hr', 'ja',
       'ca', 'et', 'pt', 'el', 'fi', 'ro', 'sv', 'af', 'da', 'tl', 'nl',
       'cy', 'sq', 'so', 'ko', 'id', 'lv', 'pl', 'sl', 'hu', 'tr', 'sw',
       'lt', 'No features in text.', 'zh-cn', 'he', 'sk', 'uk', 'bg',
       'mk'], dtype=object)

In [ ]:
music_columns_no_lg[music_columns_no_lg["language"]=="No features in text."]

,album,artist,title,language
3035,2008,nan,006,No features in text.
7742,2010,nan,007314,No features in text.


In [ ]:
import advertools as adv
adv.stopwords.keys()

ModuleNotFoundError: ignored